In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer

from sklearn import metrics


import pandas as pd
import numpy as np
import scipy as sp


from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn import cross_validation
from sklearn.svm import LinearSVC




from sklearn.metrics import classification_report
import itertools as it
import operator as op
from sklearn.feature_extraction import DictVectorizer

In [32]:
import warnings

In [27]:
from sklearn.feature_extraction import DictVectorizer

In [31]:
from sklearn.metrics import classification_report
from sklearn import cross_validation

from sklearn.svm import LinearSVC



from sklearn import metrics



In [3]:

def get_word_set(filename):
    """
    Gets the dictionary from our preprocessed dataset
    """
    with open (filename, 'r') as f:
        tmp = f.readlines()
        tmp = [line.rstrip() for line in tmp]   
    word_split = lambda x: x.split()[2].split(':')[0]
    return set(map(word_split, tmp))



In [4]:
def read_file(filename):
 
    reviews = []

    with open(filename, 'r') as f:
        t = f.readlines()
    for line in t:
        ids, rating, review = line.split(',', 2)
        reviews.append([ids, rating, review.strip().lower()])
    df = pd.DataFrame(reviews, columns=['id', 'rating', 'review'])
    df['rating'] = df.rating.astype("int")
    return df
    

def data_maker(_data):
    """
    hacky method to pivot the data from 'long' to 'wide' for each review.
    Args:
        _data: unzipped group-by'd list of stuff
    Returns:
        list of dictionaries with k: v s.t. k = word, v = count(word). 
    """
    res = []
    for row in _data:
        z = {}
        for line in row:
            k, v = line[1].split(':')
            z[k] = int(v)
        res.append(z)
    return res


def read_file(filename):
    """
    preprocesses some of the weirdness out of the format we saved to
    Returns:
        list of all lines in the files according to: [(id, rating), 'word:<count>']
    """
    with open (filename, 'r') as f:
        tmp = f.readlines()
        tmp = [line.rstrip() for line in tmp]
        split = lambda x: x.split('\t')
        tmp = list(map(split, tmp))
        tup, val = list(zip(*tmp))
        # this bit changes all the string tuples to python tuples. 
        tmp = list(zip(map(eval, tup), val))
    return tmp


def make_train_data(data):
    """ 
    Uses Sklearn's helper method to vectorize a list of dictionary features into a scipy 
    sparse matrix for training
    Returns:
        tuple of data, sparse_matrix
    """
    dv = DictVectorizer()
    # _data = dict(zip(ids, data))
    return dv.fit_transform(data)
    

get_neg_ids = lambda x: x[0][1] < 5
get_pos_ids = lambda x: x[0][1] > 5

# gets a list of sentiment scores
get_sent = lambda x: 1 if x[1] > 5 else 0

In [5]:
raw_train = read_file('./train_mr_output.csv')

In [6]:
raw_test = read_file('./test_mr_output.csv')

In [7]:
print('raw training list: ' + str(len(raw_train)))
print('raw test list: ' + str(len(raw_test)))

raw training list: 2518584
raw test list: 2464517


In [8]:
# raw_test = read_file('./test_mr_output.csv')
test_grouped = it.groupby(raw_test, op.itemgetter(0))

test_g = [(k, list(g)) for k, g in test_grouped]
test_ids, test_tmp = zip(*test_g)

test = data_maker(test_tmp)




In [9]:

train_grouped = it.groupby(raw_train, op.itemgetter(0))

train_g = [(k, list(g)) for k, g in train_grouped]
train_ids, train_tmp = zip(*train_g)

train = data_maker(train_tmp)


In [10]:
y_test = list(map(get_sent, test_ids))
y_train = list(map(get_sent, train_ids))


In [10]:
len(test)

25000

In [11]:
len(train)

25000

In [11]:
data = []
data.extend(train)
data.extend(test)

In [12]:
X = make_train_data(data)

In [14]:
X

<50000x103781 sparse matrix of type '<class 'numpy.float64'>'
	with 4983101 stored elements in Compressed Sparse Row format>

In [ ]:
t = pd.DataFrame(raw_train, columns=['id', 'val'])



x = lambda x: x.split(':')

t['word'], t['count'] = zip(*map(x, t['val']))

t = t.drop('val', axis=1)

In [13]:
X_train = X[0:25000]
X_test = X[25000:]

In [14]:
X_train

<25000x103781 sparse matrix of type '<class 'numpy.float64'>'
	with 2518584 stored elements in Compressed Sparse Row format>

In [15]:
X_test

<25000x103781 sparse matrix of type '<class 'numpy.float64'>'
	with 2464517 stored elements in Compressed Sparse Row format>

In [16]:
nb = MultinomialNB(alpha=0.3)

In [17]:
sgd = SGDClassifier(loss='log', penalty='elasticnet', alpha=0.0001, eta0=0.0001, n_iter=5, warm_start=True)

In [67]:
nb_scores = cross_validation.cross_val_score(nb, X_train, y_train, cv=10)
sgd_scores = cross_validation.cross_val_score(sgd, X_train, y_train, cv=10)

/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)
/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspe

In [68]:
print("Accuracy: %0.2f (+/- %0.2f)" % (nb_scores.mean(), nb_scores.std() * 2))
print("Accuracy: %0.2f (+/- %0.2f)" % (sgd_scores.mean(), sgd_scores.std() * 2))

Accuracy: 0.71 (+/- 0.02)
Accuracy: 0.72 (+/- 0.02)


In [18]:
nb.fit(X_train, y_train)

/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


MultinomialNB(alpha=0.3, class_prior=None, fit_prior=True)

In [19]:
sgd.fit(X_train, y_train)

/usr/local/lib/python3.5/site-packages/sklearn/base.py:175: DeprecationWarning: inspect.getargspec() is deprecated, use inspect.signature() instead
  args, varargs, kw, default = inspect.getargspec(init)


SGDClassifier(alpha=0.0001, average=False, class_weight=None, epsilon=0.1,
       eta0=0.0001, fit_intercept=True, l1_ratio=0.15,
       learning_rate='optimal', loss='log', n_iter=5, n_jobs=1,
       penalty='elasticnet', power_t=0.5, random_state=None, shuffle=True,
       verbose=0, warm_start=True)

In [20]:
print(classification_report(nb.predict(X_test), y_test))
print(classification_report(sgd.predict(X_test), y_test))
# print(classification_report(svc.predict(X_test), y_test))

             precision    recall  f1-score   support

          0       0.80      0.81      0.80     17314
          1       0.56      0.54      0.55      7686

avg / total       0.73      0.73      0.73     25000

             precision    recall  f1-score   support

          0       0.86      0.79      0.82     19041
          1       0.46      0.58      0.51      5959

avg / total       0.76      0.74      0.75     25000



In [21]:
metrics.accuracy_score(sgd.predict(X_test), y_test)

0.73816000000000004

In [22]:
metrics.confusion_matrix(nb.predict(X_test), y_test)

array([[13999,  3315],
       [ 3500,  4186]])

In [23]:
import matplotlib.pyplot as plt

def plot_confusion_matrix(cm, title='Confusion matrix', cmap=plt.cm.Blues):
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(2)
    plt.xticks(tick_marks, ['negative', 'positive'], rotation=45)
    plt.yticks(tick_marks, ['negative', 'positive'])
    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


In [24]:
# from sklearn's example


# Compute confusion matrix
cm = metrics.confusion_matrix(sgd.predict(X_test), y_test)
np.set_printoptions(precision=2)
print('Confusion matrix, without normalization')
print(cm)
plt.figure()
plot_confusion_matrix(cm)

# Normalize the confusion matrix by row (i.e by the number of samples
# in each class)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
print('Normalized confusion matrix')
print(cm_normalized)
plt.figure()
plot_confusion_matrix(cm_normalized, title='Normalized confusion matrix')
plt.savefig('cm.pdf')
plt.show()

Confusion matrix, without normalization
[[14997  4044]
 [ 2502  3457]]
Normalized confusion matrix

/usr/local/lib/python3.5/site-packages/matplotlib/tight_layout.py:225: UserWarning: tight_layout : falling back to Agg renderer
  warnings.warn("tight_layout : falling back to Agg renderer")
/usr/local/lib/python3.5/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):



[[ 0.79  0.21]
 [ 0.42  0.58]]


In [86]:
plt.savefig('cm.pdf')